In [ ]:
%pip install matplotlib
%pip install opencv-python
%pip install scipy
%pip install scipy scikit-learn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 1.0 MB/s eta 0:00:0000:010:00:01
Note: you may need to restart the kernel to use updated packages.


In [9]:
import os
import cv2
import numpy as np
from scipy.ndimage import label
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

Matplotlib is building the font cache; this may take a moment.


## ЭТО ПРОРЫВ НО Я ХОЧУ ЕЩЕ - Кластеризация работает, но на воде есть процентаж ошибок(не находит корабли)

In [ ]:
import os
import argparse
import cv2
import numpy as np
import pandas as pd
from scipy.ndimage import label

# ---------------------------------------------------------------------------
# 1. Pre‑processing
# ---------------------------------------------------------------------------

def preprocess(img: np.ndarray,
               brightness: float = 1.3,
               clahe_clip: float = 2.0,
               denoise_h: int = 20) -> np.ndarray:
    # stretch to [0,255]
    img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
    # global brightness / contrast knob
    img = cv2.convertScaleAbs(img, alpha=brightness, beta=0)
    # local contrast equalisation
    clahe = cv2.createCLAHE(clipLimit=clahe_clip, tileGridSize=(8, 8))
    img = clahe.apply(img)
    # fast non‑local means denoise (good on speckle)
    img = cv2.fastNlMeansDenoising(img, None, h=denoise_h,
                                   templateWindowSize=7, searchWindowSize=21)
    return img

# ---------------------------------------------------------------------------
# 2. Object detection on a single tile
# ---------------------------------------------------------------------------

def detect_objects_advanced(tile: np.ndarray,
                            area_thresh: float = 0.001,
                            intensity_ratio: float = 2.0):
    pre = preprocess(tile)

    # adaptive threshold tuned for speckle
    bin_img = cv2.adaptiveThreshold(pre, 255,
                                    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                    cv2.THRESH_BINARY, 51, -5)

    # morphology to remove pepper noise and fill holes
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    bin_img = cv2.morphologyEx(bin_img, cv2.MORPH_OPEN, kernel, iterations=1)
    bin_img = cv2.morphologyEx(bin_img, cv2.MORPH_CLOSE, kernel, iterations=2)

    labeled, num = label(bin_img)
    objs = []
    min_pixels = int(area_thresh * tile.shape[0] * tile.shape[1])
    background_median = np.median(tile)

    for i in range(1, num + 1):
        ys, xs = np.where(labeled == i)
        if xs.size < min_pixels:
            continue
        if tile[ys, xs].mean() < intensity_ratio * background_median:
            continue
        x1, x2 = xs.min(), xs.max()
        y1, y2 = ys.min(), ys.max()
        objs.append((x1, y1, x2, y2))
    return objs

# ---------------------------------------------------------------------------
# 3. Tiling helpers (to avoid huge memory)
# ---------------------------------------------------------------------------

def split_image(image: np.ndarray, max_tile_size=(2000, 2000)):
    h, w = image.shape
    if h <= max_tile_size[1] and w <= max_tile_size[0]:
        return [(image, 0, 0)]
    tile_w = min(w, max_tile_size[0])
    tile_h = min(h, max_tile_size[1])
    tiles = []
    for y in range(0, h, tile_h):
        for x in range(0, w, tile_w):
            tiles.append((image[y:y + tile_h, x:x + tile_w], x, y))
    return tiles

# ---------------------------------------------------------------------------
# 4. High‑level wrappers
# ---------------------------------------------------------------------------

def process_image_adv(image_path: str,
                      max_tile_size=(2000, 2000),
                      area_thresh: float = 0.001,
                      intensity_ratio: float = 2.0):
    img = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8),
                       cv2.IMREAD_GRAYSCALE)
    tiles = split_image(img, max_tile_size)
    all_objects = []
    for tile, x_off, y_off in tiles:
        objs = detect_objects_advanced(tile, area_thresh, intensity_ratio)
        for x1, y1, x2, y2 in objs:
            all_objects.append((x1 + x_off, y1 + y_off,
                                x2 + x_off, y2 + y_off))
    return all_objects


def draw_objects(image_path: str, objects, output_path: str):
    img = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8),
                       cv2.IMREAD_COLOR)
    for x1, y1, x2, y2 in objects:
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.imwrite(output_path, img)


def process_folder(input_folder: str,
                   output_folder: str,
                   max_tile_size=(2000, 2000),
                   area_thresh: float = 0.001,
                   intensity_ratio: float = 2.0):
    os.makedirs(output_folder, exist_ok=True)
    summary = []
    for fname in os.listdir(input_folder):
        if not fname.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            continue
        inp = os.path.join(input_folder, fname)
        out = os.path.join(output_folder,
                           f"{os.path.splitext(fname)[0]}_out.jpg")
        objs = process_image_adv(inp, max_tile_size,
                                 area_thresh, intensity_ratio)
        draw_objects(inp, objs, out)
        summary.append({"image": fname,
                        "objects_detected": len(objs),
                        "output_image": out})
        print(f"Processed {fname}: {len(objs)} objects -> {out}")

    # optional CSV summary
    if summary:
        df = pd.DataFrame(summary)
        df.to_csv(os.path.join(output_folder, "summary.csv"), index=False)
        print(f"CSV summary saved to {output_folder}/summary.csv")


## ПРОБА ПЕРА (Пробуем прокидывать параметры исходя из процента черноты) - Тут проблема исключительно в том, что на воде он выделяет шум как объекты с этим надо что-то придумать Решение в tune_params - по играться с параметрами 

In [ ]:
from __future__ import annotations
import os
import argparse
from typing import Tuple, List

import cv2
import numpy as np
import pandas as pd
from scipy.ndimage import label

# ---------------------------------------------------------------------------
# 1. Darkness metrics
# ---------------------------------------------------------------------------

def dark_ratio(img: np.ndarray, dark_thresh: int = 40) -> float:
    """Return fraction of pixels darker than *dark_thresh* (0‑255)."""
    return float((img < dark_thresh).sum()) / img.size


def image_dark_percent(image_path: str, dark_thresh: int = 40) -> float:
    """Decode image from disk and return percentage of dark pixels (0‑1)."""
    img = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8),
                       cv2.IMREAD_GRAYSCALE)
    return dark_ratio(img, dark_thresh)


def analyse_folder_darkness(folder: str,
                            dark_thresh: int = 40,
                            csv_path: str | None = None) -> pd.DataFrame:
    rows = []
    for fname in os.listdir(folder):
        if not fname.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            continue
        pct = image_dark_percent(os.path.join(folder, fname), dark_thresh) * 100
        rows.append({"image": fname, "dark_%": pct})
    df = pd.DataFrame(rows).sort_values("dark_%", ascending=False)
    if csv_path:
        df.to_csv(csv_path, index=False)
    return df

# ---------------------------------------------------------------------------
# 2. Pre‑processing
# ---------------------------------------------------------------------------

def preprocess(img: np.ndarray,
               brightness: float = 1.3,
               clahe_clip: float = 2.0,
               denoise_h: int = 20) -> np.ndarray:
    img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
    img = cv2.convertScaleAbs(img, alpha=brightness, beta=0)
    clahe = cv2.createCLAHE(clipLimit=clahe_clip, tileGridSize=(8, 8))
    img = clahe.apply(img)
    img = cv2.fastNlMeansDenoising(img, None, h=denoise_h,
                                   templateWindowSize=7, searchWindowSize=21)
    return img

# ---------------------------------------------------------------------------
# 3. Adaptive parameter tuning per tile
# ---------------------------------------------------------------------------

def tune_params(tile: np.ndarray,
                low: float = 0.40,
                mid: float = 0.20,
                area_min_urban: float = 0.001) -> Tuple[float, float]:
    dr = dark_ratio(tile)
    if dr > low:       # water‑heavy (mostly dark)
        return area_min_urban / 5, 1.5
    if dr > mid:       # mixed
        return area_min_urban / 2, 1.8
    return area_min_urban, 2.2  # urban / bright

# ---------------------------------------------------------------------------
# 4. Detection on a tile
# ---------------------------------------------------------------------------

def detect_objects(tile: np.ndarray,
                   area_thresh: float,
                   intensity_ratio: float) -> List[Tuple[int, int, int, int]]:
    pre = preprocess(tile)
    bin_img = cv2.adaptiveThreshold(pre, 255,
                                    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                    cv2.THRESH_BINARY, 51, -5)
    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    bin_img = cv2.morphologyEx(bin_img, cv2.MORPH_OPEN, k, iterations=1)
    bin_img = cv2.morphologyEx(bin_img, cv2.MORPH_CLOSE, k, iterations=2)

    labeled, num = label(bin_img)
    objs = []
    min_pixels = int(area_thresh * tile.shape[0] * tile.shape[1])
    bg_med = np.median(tile)

    for i in range(1, num + 1):
        ys, xs = np.where(labeled == i)
        if xs.size < min_pixels:
            continue
        if tile[ys, xs].mean() < intensity_ratio * bg_med:
            continue
        objs.append((xs.min(), ys.min(), xs.max(), ys.max()))
    return objs

# ---------------------------------------------------------------------------
# 5. Tiling helper
# ---------------------------------------------------------------------------

def split_image(image: np.ndarray, max_tile_size=(2000, 2000)):
    h, w = image.shape
    if h <= max_tile_size[1] and w <= max_tile_size[0]:
        return [(image, 0, 0)]
    tw, th = max_tile_size
    tiles = []
    for y in range(0, h, th):
        for x in range(0, w, tw):
            tiles.append((image[y:y + th, x:x + tw], x, y))
    return tiles

# ---------------------------------------------------------------------------
# 6. High‑level wrappers
# ---------------------------------------------------------------------------

def process_image(image_path: str,
                  max_tile_size=(2000, 2000),
                  low: float = 0.40,
                  mid: float = 0.20,
                  area_min: float = 0.001):
    img = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8),
                       cv2.IMREAD_GRAYSCALE)
    boxes = []
    for tile, xo, yo in split_image(img, max_tile_size):
        area_t, ratio_t = tune_params(tile, low, mid, area_min)
        for x1, y1, x2, y2 in detect_objects(tile, area_t, ratio_t):
            boxes.append((x1 + xo, y1 + yo, x2 + xo, y2 + yo))
    return boxes


def draw_objects(image_path: str, objects, output_path: str):
    img = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8),
                       cv2.IMREAD_COLOR)
    for x1, y1, x2, y2 in objects:
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.imwrite(output_path, img)


def process_folder(input_folder: str,
                   output_folder: str,
                   max_tile_size=(2000, 2000),
                   low: float = 0.40,
                   mid: float = 0.20,
                   area_min: float = 0.001):
    os.makedirs(output_folder, exist_ok=True)
    summary = []
    for fname in os.listdir(input_folder):
        if not fname.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            continue
        inp = os.path.join(input_folder, fname)

        # ---- darkness analysis ----
        pct_dark = image_dark_percent(inp) * 100
        if pct_dark > low * 100:
            scene = "water‑heavy"
        elif pct_dark > mid * 100:
            scene = "mixed"
        else:
            scene = "urban"
        print(f"{fname}: dark {pct_dark:.1f}% → {scene} params")

        # ---- detection ----
        boxes = process_image(inp, max_tile_size, low, mid, area_min)
        out = os.path.join(output_folder, f"{os.path.splitext(fname)[0]}_out.jpg")
        draw_objects(inp, boxes, out)
        summary.append({"image": fname,
                        "dark_%": pct_dark,
                        "objects": len(boxes),
                        "scene": scene,
                        "output": out})

    if summary:
        pd.DataFrame(summary).to_csv(os.path.join(output_folder, "summary.csv"),
                                     index=False)
        print("CSV summary saved → summary.csv")

# ---------------------------------------------------------------------------
# 7. CLI (defaults let you run with no flags)
# ---------------------------------------------------------------------------

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Context‑aware SAR detector")
    parser.add_argument("--input", default="./images", help="Input folder with SAR images")
    parser.add_argument("--output", default="./out", help="Output folder for results")
    parser.add_argument("--tile", type=int, default=2000, help="Max tile size (pixels)")
    parser.add_argument("--low", type=float, default=0.40, help="Dark‑pixel ratio split for water‑heavy scene")
    parser.add_argument("--mid", type=float, default=0.20, help="Dark‑pixel ratio split for mixed scene")
    parser.add_argument("--area_min", type=float, default=0.001, help="Min blob area fraction for urban tiles")
    parser.add_argument("--brightness", type=float, default=1.3, help="Global brightness gain (alpha)")
    parser.add_argument("--denoise", type=int, default=20, help="fastNlMeans strength (higher = smoother)")
    args, _ = parser.parse_known_args()  # ignore extra args (e.g., Jupyter "-f")

    # patch preprocess defaults according to CLI knobs
    preprocess.__defaults__ = (args.brightness, 2.0, args.denoise)

    process_folder(args.input, args.output,
                   max_tile_size=(args.tile, args.tile),
                   low=args.low, mid=args.mid, area_min=args.area_min)


8.jpg: dark 63.9% → water‑heavy params
9.jpg: dark 96.1% → water‑heavy params
4.jpg: dark 34.6% → mixed params
5.jpg: dark 56.6% → water‑heavy params
7.jpg: dark 31.3% → mixed params
6.jpg: dark 71.4% → water‑heavy params
2.jpg: dark 61.4% → water‑heavy params
3.jpg: dark 33.7% → mixed params
1.jpg: dark 68.9% → water‑heavy params
CSV summary saved → summary.csv


In [98]:
input_folder = "images"
output_folder = "output"
process_folder(input_folder, output_folder)

8.jpg: dark 63.9% → water‑heavy params
9.jpg: dark 96.1% → water‑heavy params
4.jpg: dark 34.6% → mixed params
5.jpg: dark 56.6% → water‑heavy params
7.jpg: dark 31.3% → mixed params
6.jpg: dark 71.4% → water‑heavy params
2.jpg: dark 61.4% → water‑heavy params
3.jpg: dark 33.7% → mixed params
1.jpg: dark 68.9% → water‑heavy params
CSV summary saved → summary.csv


## Без шума но очень криво

In [60]:

def split_image(image, max_tile_size=(2000, 2000), overlap=100):
    h, w = image.shape
    if h <= max_tile_size[1] and w <= max_tile_size[0]:
        return [(image, 0, 0)]
    
    tile_size_x = min(w, max_tile_size[0])
    tile_size_y = min(h, max_tile_size[1])
    tiles = []
    
    for y in range(0, h, tile_size_y - overlap):
        for x in range(0, w, tile_size_x - overlap):
            tile = image[y:y + tile_size_y, x:x + tile_size_x]
            tiles.append((tile, x, y))
    
    return tiles

def detect_objects(image):
    image = cv2.medianBlur(image, 5)
    _, binary = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))
    
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    objects = []
    
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        objects.append((x, y, x + w, y + h))
    
    return objects

def process_image(image_path, max_tile_size=(2000, 2000)):
    image = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8), cv2.IMREAD_GRAYSCALE)
    tiles = split_image(image, max_tile_size)
    all_objects = []
    
    for tile, x_offset, y_offset in tiles:
        objects = detect_objects(tile)
        adjusted_objects = [(x + x_offset, y + y_offset, x_max + x_offset, y_max + y_offset) 
                            for x, y, x_max, y_max in objects]
        all_objects.extend(adjusted_objects)
    
    return all_objects

def draw_objects(image_path, objects, output_path):
    image = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8), cv2.IMREAD_COLOR)
    for x1, y1, x2, y2 in objects:
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.imwrite(output_path, image)

def process_folder(input_folder, output_folder, max_tile_size=(2000, 2000)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_out.bmp")
            
            objects = process_image(input_path, max_tile_size)
            draw_objects(input_path, objects, output_path)
            print(f"Processed: {filename} -> {output_path}")

## Что то неплохое 

In [61]:

def split_image(image, max_tile_size=(2000, 2000), overlap=100):
    h, w = image.shape
    if h <= max_tile_size[1] and w <= max_tile_size[0]:
        return [(image, 0, 0)]
    
    tile_size_x = min(w, max_tile_size[0])
    tile_size_y = min(h, max_tile_size[1])
    tiles = []
    
    for y in range(0, h, tile_size_y - overlap):
        for x in range(0, w, tile_size_x - overlap):
            tile = image[y:y + tile_size_y, x:x + tile_size_x]
            tiles.append((tile, x, y))
    
    return tiles

def detect_objects(image):
    image = cv2.medianBlur(image, 5)
    _, binary = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))
    
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    bright_points = []
    
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        bright_points.append((x + w // 2, y + h // 2))
    
    if bright_points:
        bright_points = np.array(bright_points)
        clustering = DBSCAN(eps=20, min_samples=2).fit(bright_points)
        labels = clustering.labels_
        
        clusters = {}
        for i, label in enumerate(labels):
            if label != -1:
                if label not in clusters:
                    clusters[label] = []
                clusters[label].append(bright_points[i])
        
        objects = []
        for cluster in clusters.values():
            cluster = np.array(cluster)
            x_min, y_min = cluster.min(axis=0)
            x_max, y_max = cluster.max(axis=0)
            objects.append((x_min, y_min, x_max, y_max))
        
        return objects
    
    return []

def process_image(image_path, max_tile_size=(2000, 2000)):
    image = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8), cv2.IMREAD_GRAYSCALE)
    tiles = split_image(image, max_tile_size)
    all_objects = []
    
    for tile, x_offset, y_offset in tiles:
        objects = detect_objects(tile)
        adjusted_objects = [(x + x_offset, y + y_offset, x_max + x_offset, y_max + y_offset) 
                            for x, y, x_max, y_max in objects]
        all_objects.extend(adjusted_objects)
    
    return all_objects

def draw_objects(image_path, objects, output_path):
    image = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8), cv2.IMREAD_COLOR)
    for x1, y1, x2, y2 in objects:
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.imwrite(output_path, image)

def process_folder(input_folder, output_folder, max_tile_size=(2000, 2000)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_out.bmp")
            
            objects = process_image(input_path, max_tile_size)
            draw_objects(input_path, objects, output_path)
            print(f"Processed: {filename} -> {output_path}")

In [62]:
# Функция для рисования объектов (прямоугольников) на изображении
def draw_objects(image_path, objects, output_path):
    image = cv2.imdecode(np.fromfile(image_path, dtype=np.uint8), cv2.IMREAD_COLOR)
    for x1, y1, x2, y2 in objects:
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.imwrite(output_path, image)

# Функция для обработки изображения и извлечения объектов
def process_image(image_path, n_clusters=3):
    # 1. Загрузка данных
    sar_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if sar_image is None:
        raise ValueError(f"Не удалось загрузить файл: {image_path}")

    # 2. Предобработка
    denoised_image = cv2.medianBlur(sar_image, 5)
    normalized_image = (denoised_image - np.min(denoised_image)) / (np.max(denoised_image) - np.min(denoised_image))

    # 3. Кластеризация
    pixels = normalized_image.reshape(-1, 1)
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(pixels)
    labels = kmeans.labels_.reshape(normalized_image.shape)

    # 4. Постобработка и извлечение объектов
    objects = []
    for cluster_id in range(1, n_clusters):  # Пропускаем кластер 0 (фон)
        binary_mask = (labels == cluster_id).astype(np.uint8)
        binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_OPEN, np.ones((5, 5), np.uint8))

        # Находим контуры объектов
        contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for contour in contours:
            if cv2.contourArea(contour) > 100:  # Фильтр по минимальной площади
                x, y, w, h = cv2.boundingRect(contour)
                objects.append((x, y, x + w, y + h))

    return objects

# Основная функция для обработки одного файла
def process_and_visualize(image_path, output_path, n_clusters=3):
    # Обработка изображения и получение объектов
    objects = process_image(image_path, n_clusters=n_clusters)
    
    # Рисование объектов на изображении
    draw_objects(image_path, objects, output_path)
    print(f"Processed: {image_path} -> {output_path}")

    # Визуализация (опционально)
    sar_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    segmented_image = cv2.imread(output_path, cv2.IMREAD_GRAYSCALE)

    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.title("Исходное изображение")
    plt.imshow(sar_image, cmap="gray")
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.title("Сегментация с объектами")
    plt.imshow(cv2.cvtColor(cv2.imread(output_path), cv2.COLOR_BGR2RGB))
    plt.axis("off")

    plt.tight_layout()
    plt.show()

# Пример использования
if __name__ == "__main__":
    image_path = "sar_image.bmp"  # Укажите путь к вашему .bmp файлу
    output_path = "segmented_sar_image_with_boxes.bmp"
    process_and_visualize(image_path, output_path, n_clusters=3)

[ WARN:0@2300.392] global loadsave.cpp:268 findDecoder imread_('sar_image.bmp'): can't open/read file: check file path/integrity


ValueError: Не удалось загрузить файл: sar_image.bmp

## Нейронка попробовал DeepLabV3+ она нам не подходит.

In [91]:
input_folder = "images"
output_folder = "output"
process_folder(input_folder, output_folder)

Processed 8.jpg: 3 objects -> output/8_out.jpg
Processed 9.jpg: 1 objects -> output/9_out.jpg
Processed 4.jpg: 17 objects -> output/4_out.jpg
Processed 5.jpg: 10 objects -> output/5_out.jpg
Processed 7.jpg: 14 objects -> output/7_out.jpg
Processed 6.jpg: 11 objects -> output/6_out.jpg
Processed 2.jpg: 13 objects -> output/2_out.jpg
Processed 3.jpg: 4 objects -> output/3_out.jpg
Processed 1.jpg: 15 objects -> output/1_out.jpg
CSV summary saved to output/summary.csv
